In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//span[@data-ui-id="page-title-wrapper"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                sku = html.xpath('//div[@class="product attribute sku"]/div[@class="value"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//meta[@property="og:image"]/@content')[0].strip()
                
                # = = = = = = = = = = = = = = =
                                
                detail = ''
                list_detail = [li.strip() for li in html.xpath('//div[@id="product_details"]/descendant::li/text()')]
                for i in range(len(list_detail)):
                    detail += str(i+1) + '. ' + list_detail[i] + '\n'
                detail = detail.strip()
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//div[@id="specifications"]/descendant::tr')
                
                # = = = = = = = = = = = = = = =

                list_td = html.xpath('//div[@id="cross_references"]/descendant::tr/td')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Name': input_.loc[a, 'Name'],
                                         'Title': title,
                                         'Sku': sku,
                                         'Detail': detail,
                                         'Vehicle': '',
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])
                
                for td in list_td:
                    df_temp.loc[0, 'OE_' + td.xpath('./text()')[0].split(' ')[0].strip()] = ' '.join([text.strip() for text in td.xpath('./text()')[0].split(' ')[1:]])

                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./td[1]/text()')[0].strip().lower()] = tr.xpath('./td[2]/text()')[0].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：128

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30578/s/spa-2395-essential-filter/
[尝试次数：1] - [剩余数量：108] - [当前时间：16:17:27]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30589/s/spa-2498-essential-filter/
[尝试次数：1] - [剩余数量：107] - [当前时间：16:17:27]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30602/s/spc-2022-essential-filter/
[尝试次数：1] - [剩余数量：106] - [当前时间：16:17:27]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30599/s/spc-2005-essential-filter/
[尝试次数：1] - [剩余数量：105] - [当前时间：16:17:27]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30597/s/spc-2002-essential-filter/
[尝试次数：1] - [剩余数量：104] - [当前时间：16:17:27]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30596/s/spc-2001-essential-filter/
[尝试次数：1] - [剩余数量：103] - [当前时间：16:17:27]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30601/s/spc-2013-essential-filter/
[尝试次数：1] - [剩余数量：102] - [当前时间：16:17:28

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30550/s/spa-2144-essential-filter/
[尝试次数：1] - [剩余数量：50] - [当前时间：16:17:35]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30555/s/spa-2206-essential-filter/
[尝试次数：1] - [剩余数量：49] - [当前时间：16:17:35]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30539/s/spa-0945-essential-filter/
[尝试次数：1] - [剩余数量：48] - [当前时间：16:17:35]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30541/s/spa-2012-essential-filter/
[尝试次数：1] - [剩余数量：47] - [当前时间：16:17:35]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30546/s/spa-2106-essential-filter/
[尝试次数：1] - [剩余数量：46] - [当前时间：16:17:35]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30548/s/spa-2129-essential-filter/
[尝试次数：1] - [剩余数量：45] - [当前时间：16:17:35]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30567/s/spa-2309-essential-filter/
[尝试次数：1] - [剩余数量：44] - [当前时间：16:17:35]

[ok] - https:

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30573/s/spa-2370-essential-filter/
[尝试次数：1] - [剩余数量：0] - [当前时间：16:17:42]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30561/s/spa-2276-essential-filter/
[尝试次数：1] - [剩余数量：0] - [当前时间：16:17:42]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30560/s/spa-2260-essential-filter/
[尝试次数：1] - [剩余数量：0] - [当前时间：16:17:42]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30562/s/spa-2281-essential-filter/
[尝试次数：1] - [剩余数量：0] - [当前时间：16:17:43]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30564/s/spa-2287-essential-filter/
[尝试次数：1] - [剩余数量：0] - [当前时间：16:17:43]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30557/s/spa-2247-essential-filter/
[尝试次数：1] - [剩余数量：0] - [当前时间：16:17:43]

[ok] - https://www.spectreperformance.com/catalog/product/view/id/30558/s/spa-2249-essential-filter/
[尝试次数：1] - [剩余数量：0] - [当前时间：16:17:43]

[ok] - https://www.s